In [30]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import statsmodels.formula.api as smf

%matplotlib inline

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [8]:
#get data
location = "../Python Learning/datasets/DC_Properties.csv"

dc = pd.read_csv(location)
dc.head()

C:\Users\Charlie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,...,LONGITUDE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT
0,0,4,0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,...,-77.040832,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
1,1,3,1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,...,-77.040764,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
2,2,3,1,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,...,-77.040678,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
3,3,3,1,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,...,-77.040629,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
4,4,2,1,Warm Cool,Y,1.0,11,3,1913.0,2012.0,...,-77.039361,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW


In [10]:
#list columns
dc.columns

Index(['Unnamed: 0', 'BATHRM', 'HF_BATHRM', 'HEAT', 'AC', 'NUM_UNITS', 'ROOMS',
       'BEDRM', 'AYB', 'YR_RMDL', 'EYB', 'STORIES', 'SALEDATE', 'PRICE',
       'QUALIFIED', 'SALE_NUM', 'GBA', 'BLDG_NUM', 'STYLE', 'STRUCT', 'GRADE',
       'CNDTN', 'EXTWALL', 'ROOF', 'INTWALL', 'KITCHENS', 'FIREPLACES',
       'USECODE', 'LANDAREA', 'GIS_LAST_MOD_DTTM', 'SOURCE', 'CMPLX_NUM',
       'LIVING_GBA', 'FULLADDRESS', 'CITY', 'STATE', 'ZIPCODE', 'NATIONALGRID',
       'LATITUDE', 'LONGITUDE', 'ASSESSMENT_NBHD', 'ASSESSMENT_SUBNBHD',
       'CENSUS_TRACT', 'CENSUS_BLOCK', 'WARD', 'SQUARE', 'X', 'Y', 'QUADRANT'],
      dtype='object')

In [13]:
#remove some columns
cleandc = dc.drop(['Unnamed: 0', 'EXTWALL', 'ROOF', 'INTWALL', 'GIS_LAST_MOD_DTTM', 'SOURCE', 'CMPLX_NUM',
       'LIVING_GBA', 'NATIONALGRID', 'LATITUDE', 'LONGITUDE', 'ASSESSMENT_NBHD', 'ASSESSMENT_SUBNBHD',
       'CENSUS_TRACT', 'CENSUS_BLOCK', 'WARD', 'SQUARE', 'X', 'Y',], axis = 1)

In [16]:
#find sum of null values in columns
cleandc.isnull().sum()

BATHRM             0
HF_BATHRM          0
HEAT               0
AC                 0
NUM_UNITS      52261
ROOMS              0
BEDRM              0
AYB              271
YR_RMDL        78029
EYB                0
STORIES        52305
SALEDATE       26770
PRICE          60741
QUALIFIED          0
SALE_NUM           0
GBA            52261
BLDG_NUM           0
STYLE          52261
STRUCT         52261
GRADE          52261
CNDTN          52261
KITCHENS       52262
FIREPLACES         0
USECODE            0
LANDAREA           0
FULLADDRESS    52917
CITY           52906
STATE          52906
ZIPCODE            1
QUADRANT         237
dtype: int64

In [27]:
#remove all null values to clean and decrease data
cleandc = cleandc.dropna()

In [28]:
#verify last code
cleandc.isnull().sum()

BATHRM         0
HF_BATHRM      0
HEAT           0
AC             0
NUM_UNITS      0
ROOMS          0
BEDRM          0
AYB            0
YR_RMDL        0
EYB            0
STORIES        0
SALEDATE       0
PRICE          0
QUALIFIED      0
SALE_NUM       0
GBA            0
BLDG_NUM       0
STYLE          0
STRUCT         0
GRADE          0
CNDTN          0
KITCHENS       0
FIREPLACES     0
USECODE        0
LANDAREA       0
FULLADDRESS    0
CITY           0
STATE          0
ZIPCODE        0
QUADRANT       0
dtype: int64

In [29]:
#get statistics of dataframe
cleandc.describe()

,BATHRM,HF_BATHRM,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,EYB,STORIES,PRICE,SALE_NUM,GBA,BLDG_NUM,KITCHENS,FIREPLACES,USECODE,LANDAREA,ZIPCODE
count,32979.000000,32979.000000,32979.000000,32979.000000,32979.000000,32979.000000,32979.000000,32979.000000,32979.000000,3.297900e+04,32979.000000,32979.000000,32979.000000,32979.000000,32979.000000,32979.000000,32979.000000,32979.000000
mean,2.375663,0.682798,1.239759,7.575336,3.526062,1924.151642,2004.242639,1969.399618,2.159298,6.854156e+05,2.043240,1784.728918,1.000121,1.283423,0.758907,13.586889,3150.510780,20010.235028
std,1.056258,0.597738,0.612434,2.329215,1.167389,22.193923,16.679813,10.527042,5.212966,6.434792e+05,1.631814,858.534784,0.011013,0.672215,0.985923,4.526610,3280.832554,7.510712
min,0.000000,0.000000,0.000000,0.000000,0.000000,1754.000000,20.000000,1915.000000,0.000000,1.000000e+00,1.000000,407.000000,1.000000,0.000000,0.000000,11.000000,216.000000,20001.000000
25%,2.000000,0.000000,1.000000,6.000000,3.000000,1910.000000,2002.000000,1964.000000,2.000000,3.250000e+05,1.000000,1240.000000,1.000000,1.000000,0.000000,11.000000,1468.000000,20003.000000
50%,2.000000,1.000000,1.000000,7.000000,3.000000,1924.000000,2008.000000,1967.000000,2.000000,5.680000e+05,1.000000,1560.000000,1.000000,1.000000,1.000000,12.000000,2085.000000,20010.000000
75%,3.000000,1.000000,1.000000,9.000000,4.000000,1939.000000,2013.000000,1972.000000,2.000000,8.450000e+05,3.000000,2086.000000,1.000000,1.000000,1.000000,13.000000,3975.000000,20016.000000
max,11.000000,11.000000,6.000000,31.000000,20.000000,2015.000000,2019.000000,2017.000000,826.000000,2.396029e+07,15.000000,15902.000000,2.000000,44.000000,13.000000,39.000000,155905.000000,20052.000000


In [32]:
#Linear regression
statsdc = smf.ols('PRICE ~ FIREPLACES + LANDAREA + YR_RMDL', data = cleandc).fit()
statsdc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PRICE   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     5594.
Date:                Sun, 20 Jan 2019   Prob (F-statistic):               0.00
Time:                        21:57:29   Log-Likelihood:            -4.8109e+05
No. Observations:               32979   AIC:                         9.622e+05
Df Residuals:                   32975   BIC:                         9.622e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -6.879e+06   3.48e+05    -19.760      0.000   -7.56e+06    -6.2e+06
FIREPLACES  2.835e+05   3097.232     91.537      0.000    2.77e+05     2.9e+05
LANDAREA      52.9069      0.927     57.046      0.000      51.089      54.725
YR_RMDL     3583.5909    173.606     20.642      0.000    3243.317    3923.865
==============================================================================
Omnibus:                    39353.682   Durbin-Watson:                   1.353
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         18448609.002
Skew:                           5.861   Prob(JB):                         0.00
Kurtosis:                     118.275   Cond. No.                     5.76e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.76e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
#Split data
y = cleandc['PRICE']
x = cleandc.drop(['PRICE', 'QUADRANT'], axis = 1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 15)

In [41]:
#Logistic Regression
statsdc2 = LogisticRegression()
#cannot fit logreg
#statsdc2.fit(x_train, y_train)

In [42]:
statsdc2.score(x_train, y_train)

NotFittedError: This LogisticRegression instance is not fitted yet

In [43]:
y_pred = statsdc2.predict(x_test)

NotFittedError: This LogisticRegression instance is not fitted yet

In [45]:
model = tree.DecisionTreeClassifier()

model.fit(x_train, y_train)

ValueError: could not convert string to float: 'Ht Pump'